# Chapter 11: Server Actions

Server Actions are one of Next.js's most powerful features, allowing you to run server-side code directly from your components. They eliminate the need to create separate API routes for mutations, simplify form handling, and provide a seamless way to update data while automatically handling revalidation.

By the end of this chapter, you'll master creating Server Actions, handling complex form submissions, implementing secure mutation patterns, and building robust data modification flows.

## 11.1 Introduction to Server Actions

Server Actions are asynchronous functions that execute on the server. They can be called from Client Components or Server Components, making them incredibly versatile for handling data mutations.

### Why Server Actions Matter

**Before Server Actions:**
```tsx
// Traditional approach - Separate API route + Client Component

// app/api/contact/route.ts
export async function POST(request: Request) {
  const data = await request.json();
  // Process data...
  return Response.json({ success: true });
}

// components/ContactForm.tsx
"use client";
export function ContactForm() {
  const handleSubmit = async (e: FormEvent) => {
    e.preventDefault();
    await fetch('/api/contact', { method: 'POST', body: JSON.stringify(data) });
  };
}
```

**With Server Actions:**
```tsx
// Single file - Server Action + Component

// app/actions/contact.ts
'use server';
export async function submitContactForm(data: FormData) {
  // Runs on server - no API route needed!
  await saveToDatabase(data);
}

// components/ContactForm.tsx
"use client";
import { submitContactForm } from './actions';
export function ContactForm() {
  return <form action={submitContactForm}>...</form>;
}
```

### Key Benefits

1. **Reduced Boilerplate**: No need to create API routes for every mutation
2. **Type Safety**: Full TypeScript support across client-server boundary
3. **Progressive Enhancement**: Forms work without JavaScript, enhanced with JS
4. **Automatic Revalidation**: Built-in cache revalidation after mutations
5. **Security**: Server code never exposed to client
6. **Performance**: Reduced network overhead compared to API routes

## 11.2 Creating Server Actions

Server Actions are created using the `'use server'` directive. They can be defined in separate files or inline within components.

### Basic Server Action Structure

```tsx
// app/actions/user.ts
'use server';

import { revalidatePath } from 'next/cache';
import { redirect } from 'next/navigation';

interface CreateUserInput {
  name: string;
  email: string;
  password: string;
}

export async function createUser(formData: FormData) {
  // This code runs on the server
  const name = formData.get('name') as string;
  const email = formData.get('email') as string;
  const password = formData.get('password') as string;

  // Validate input
  if (!name || !email || !password) {
    throw new Error('Missing required fields');
  }

  // Check if user exists
  const existingUser = await db.user.findUnique({ where: { email } });
  if (existingUser) {
    throw new Error('User already exists');
  }

  // Hash password
  const hashedPassword = await bcrypt.hash(password, 10);

  // Create user
  const user = await db.user.create({
    data: {
      name,
      email,
      password: hashedPassword,
    },
  });

  // Revalidate relevant paths
  revalidatePath('/users');
  
  // Redirect after success
  redirect(`/users/${user.id}`);
}
```

### Inline Server Actions

You can also define Server Actions directly in components (Server Components only):

```tsx
// app/posts/new/page.tsx
import { revalidatePath } from 'next/cache';
import { redirect } from 'next/navigation';

export default function NewPostPage() {
  // Inline Server Action
  async function createPost(formData: FormData) {
    'use server';
    
    const title = formData.get('title') as string;
    const content = formData.get('content') as string;
    
    await db.post.create({
      data: { title, content, published: false }
    });
    
    revalidatePath('/posts');
    redirect('/posts');
  }

  return (
    <form action={createPost}>
      <input name="title" placeholder="Title" required />
      <textarea name="content" placeholder="Content" required />
      <button type="submit">Create Post</button>
    </form>
  );
}
```

### Server Actions with Parameters

Pass additional parameters to Server Actions using closures:

```tsx
// components/UpdateUserButton.tsx
'use server';

import { updateUserRole } from '@/app/actions/user';

interface UpdateUserButtonProps {
  userId: string;
  currentRole: string;
}

export function UpdateUserButton({ userId, currentRole }: UpdateUserButtonProps) {
  // Create Server Action with bound parameters
  async function handleUpdateRole(formData: FormData) {
    'use server';
    
    const newRole = formData.get('role') as string;
    
    await db.user.update({
      where: { id: userId },
      data: { role: newRole }
    });
    
    revalidatePath('/users');
  }

  return (
    <form action={handleUpdateRole}>
      <select name="role" defaultValue={currentRole}>
        <option value="user">User</option>
        <option value="admin">Admin</option>
        <option value="editor">Editor</option>
      </select>
      <button type="submit">Update Role</button>
    </form>
  );
}
```

## 11.3 Form Handling with Server Actions

Server Actions shine when handling forms, providing progressive enhancement out of the box.

### Basic Form Handling

```tsx
// app/contact/page.tsx
'use client';

import { useState } from 'react';
import { submitContactForm } from '@/app/actions/contact';

export default function ContactPage() {
  const [status, setStatus] = useState<'idle' | 'success' | 'error'>('idle');

  async function handleSubmit(formData: FormData) {
    try {
      await submitContactForm(formData);
      setStatus('success');
    } catch (error) {
      setStatus('error');
    }
  }

  return (
    <div className="container mx-auto px-4 py-8 max-w-2xl">
      <h1 className="text-3xl font-bold mb-8">Contact Us</h1>

      {status === 'success' && (
        <div className="bg-green-50 text-green-800 p-4 rounded-lg mb-6">
          Thank you! We'll get back to you soon.
        </div>
      )}

      {status === 'error' && (
        <div className="bg-red-50 text-red-800 p-4 rounded-lg mb-6">
          Something went wrong. Please try again.
        </div>
      )}

      <form action={handleSubmit} className="space-y-6">
        <div>
          <label htmlFor="name" className="block text-sm font-medium text-gray-700 mb-2">
            Name
          </label>
          <input
            type="text"
            id="name"
            name="name"
            required
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
          />
        </div>

        <div>
          <label htmlFor="email" className="block text-sm font-medium text-gray-700 mb-2">
            Email
          </label>
          <input
            type="email"
            id="email"
            name="email"
            required
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
          />
        </div>

        <div>
          <label htmlFor="message" className="block text-sm font-medium text-gray-700 mb-2">
            Message
          </label>
          <textarea
            id="message"
            name="message"
            rows={5}
            required
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
          />
        </div>

        <button
          type="submit"
          className="w-full bg-blue-600 text-white py-3 rounded-lg hover:bg-blue-700 transition-colors"
        >
          Send Message
        </button>
      </form>
    </div>
  );
}
```

### Form Validation with Server Actions

Implement comprehensive validation:

```tsx
// app/actions/contact.ts
'use server';

import { z } from 'zod';

// Define validation schema
const contactSchema = z.object({
  name: z.string().min(2, 'Name must be at least 2 characters'),
  email: z.string().email('Invalid email address'),
  subject: z.string().min(5, 'Subject must be at least 5 characters'),
  message: z.string().min(10, 'Message must be at least 10 characters'),
  priority: z.enum(['low', 'medium', 'high']),
});

export type ContactFormState = {
  errors?: {
    name?: string[];
    email?: string[];
    subject?: string[];
    message?: string[];
    priority?: string[];
  };
  message?: string;
  success?: boolean;
};

export async function submitContactForm(
  prevState: ContactFormState,
  formData: FormData
): Promise<ContactFormState> {
  // Validate form data
  const validatedFields = contactSchema.safeParse({
    name: formData.get('name'),
    email: formData.get('email'),
    subject: formData.get('subject'),
    message: formData.get('message'),
    priority: formData.get('priority'),
  });

  if (!validatedFields.success) {
    return {
      errors: validatedFields.error.flatten().fieldErrors,
      message: 'Please correct the errors below.',
    };
  }

  try {
    // Save to database or send email
    await saveContactForm(validatedFields.data);
    
    return {
      success: true,
      message: 'Thank you for your message! We will respond within 24 hours.',
    };
  } catch (error) {
    return {
      message: 'An error occurred while submitting the form. Please try again.',
    };
  }
}
```

```tsx
// app/contact/page.tsx
'use client';

import { useFormState, useFormStatus } from 'react-dom';
import { submitContactForm } from '@/app/actions/contact';

// Submit button with loading state
function SubmitButton() {
  const { pending } = useFormStatus();
  
  return (
    <button
      type="submit"
      disabled={pending}
      className="w-full bg-blue-600 text-white py-3 rounded-lg hover:bg-blue-700 disabled:opacity-50 disabled:cursor-not-allowed transition-colors"
    >
      {pending ? 'Sending...' : 'Send Message'}
    </button>
  );
}

export default function ContactPage() {
  const [state, formAction] = useFormState(submitContactForm, {});

  return (
    <div className="container mx-auto px-4 py-8 max-w-2xl">
      <h1 className="text-3xl font-bold mb-8">Contact Us</h1>

      {state.message && (
        <div className={`p-4 rounded-lg mb-6 ${
          state.success ? 'bg-green-50 text-green-800' : 'bg-red-50 text-red-800'
        }`}>
          {state.message}
        </div>
      )}

      <form action={formAction} className="space-y-6">
        <div>
          <label htmlFor="name" className="block text-sm font-medium text-gray-700 mb-2">
            Name
          </label>
          <input
            type="text"
            id="name"
            name="name"
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500"
          />
          {state.errors?.name && (
            <p className="mt-1 text-sm text-red-600">{state.errors.name[0]}</p>
          )}
        </div>

        <div>
          <label htmlFor="email" className="block text-sm font-medium text-gray-700 mb-2">
            Email
          </label>
          <input
            type="email"
            id="email"
            name="email"
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500"
          />
          {state.errors?.email && (
            <p className="mt-1 text-sm text-red-600">{state.errors.email[0]}</p>
          )}
        </div>

        <div>
          <label htmlFor="subject" className="block text-sm font-medium text-gray-700 mb-2">
            Subject
          </label>
          <input
            type="text"
            id="subject"
            name="subject"
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500"
          />
          {state.errors?.subject && (
            <p className="mt-1 text-sm text-red-600">{state.errors.subject[0]}</p>
          )}
        </div>

        <div>
          <label htmlFor="priority" className="block text-sm font-medium text-gray-700 mb-2">
            Priority
          </label>
          <select
            id="priority"
            name="priority"
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500"
          >
            <option value="low">Low</option>
            <option value="medium">Medium</option>
            <option value="high">High</option>
          </select>
        </div>

        <div>
          <label htmlFor="message" className="block text-sm font-medium text-gray-700 mb-2">
            Message
          </label>
          <textarea
            id="message"
            name="message"
            rows={5}
            className="w-full px-4 py-2 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500"
          />
          {state.errors?.message && (
            <p className="mt-1 text-sm text-red-600">{state.errors.message[0]}</p>
          )}
        </div>

        <SubmitButton />
      </form>
    </div>
  );
}
```

### File Uploads with Server Actions

Handle file uploads securely:

```tsx
// app/actions/upload.ts
'use server';

import { writeFile } from 'fs/promises';
import { join } from 'path';
import { v4 as uuidv4 } from 'uuid';

export async function uploadImage(formData: FormData) {
  const file = formData.get('image') as File;
  
  if (!file) {
    throw new Error('No file uploaded');
  }

  // Validate file type
  const allowedTypes = ['image/jpeg', 'image/png', 'image/webp'];
  if (!allowedTypes.includes(file.type)) {
    throw new Error('Invalid file type. Only JPEG, PNG and WebP allowed.');
  }

  // Validate file size (5MB max)
  const maxSize = 5 * 1024 * 1024;
  if (file.size > maxSize) {
    throw new Error('File too large. Maximum size is 5MB.');
  }

  // Generate unique filename
  const bytes = await file.arrayBuffer();
  const buffer = Buffer.from(bytes);
  const filename = `${uuidv4()}-${file.name}`;
  const path = join(process.cwd(), 'public', 'uploads', filename);
  
  // Save file
  await writeFile(path, buffer);
  
  // Return public URL
  return `/uploads/${filename}`;
}
```

```tsx
// components/ImageUploader.tsx
'use client';

import { useState } from 'react';
import { uploadImage } from '@/app/actions/upload';

export function ImageUploader() {
  const [preview, setPreview] = useState<string | null>(null);
  const [uploading, setUploading] = useState(false);
  const [uploadedUrl, setUploadedUrl] = useState<string | null>(null);

  async function handleSubmit(formData: FormData) {
    setUploading(true);
    try {
      const url = await uploadImage(formData);
      setUploadedUrl(url);
      setPreview(null);
    } catch (error) {
      alert(error instanceof Error ? error.message : 'Upload failed');
    } finally {
      setUploading(false);
    }
  }

  function handleFileChange(e: React.ChangeEvent<HTMLInputElement>) {
    const file = e.target.files?.[0];
    if (file) {
      const reader = new FileReader();
      reader.onloadend = () => setPreview(reader.result as string);
      reader.readAsDataURL(file);
    }
  }

  return (
    <div className="space-y-4">
      <form action={handleSubmit} className="space-y-4">
        <input
          type="file"
          name="image"
          accept="image/*"
          onChange={handleFileChange}
          required
          className="block w-full text-sm text-gray-500 file:mr-4 file:py-2 file:px-4 file:rounded-full file:border-0 file:text-sm file:font-semibold file:bg-blue-50 file:text-blue-700 hover:file:bg-blue-100"
        />
        
        {preview && (
          <div className="relative w-64 h-64">
            <img src={preview} alt="Preview" className="w-full h-full object-cover rounded-lg" />
          </div>
        )}
        
        <button
          type="submit"
          disabled={uploading}
          className="px-4 py-2 bg-blue-600 text-white rounded-lg disabled:opacity-50"
        >
          {uploading ? 'Uploading...' : 'Upload Image'}
        </button>
      </form>

      {uploadedUrl && (
        <div className="p-4 bg-green-50 rounded-lg">
          <p className="text-green-800">Upload successful!</p>
          <img src={uploadedUrl} alt="Uploaded" className="mt-2 w-32 h-32 object-cover rounded" />
        </div>
      )}
    </div>
  );
}
```

## 11.4 Mutation Patterns

Server Actions support all CRUD operations with clean, reusable patterns.

### Create Operations

```tsx
// app/actions/todo.ts
'use server';

import { revalidatePath } from 'next/cache';
import { z } from 'zod';

const createTodoSchema = z.object({
  title: z.string().min(1, 'Title is required'),
  description: z.string().optional(),
  priority: z.enum(['low', 'medium', 'high']).default('medium'),
  dueDate: z.string().optional(),
});

export async function createTodo(formData: FormData) {
  const validated = createTodoSchema.parse({
    title: formData.get('title'),
    description: formData.get('description'),
    priority: formData.get('priority'),
    dueDate: formData.get('dueDate'),
  });

  await db.todo.create({
    data: {
      ...validated,
      completed: false,
      createdAt: new Date(),
    },
  });

  revalidatePath('/todos');
}
```

### Update Operations

```tsx
// app/actions/todo.ts
'use server';

export async function updateTodo(todoId: string, formData: FormData) {
  const title = formData.get('title') as string;
  const completed = formData.get('completed') === 'on';
  
  await db.todo.update({
    where: { id: todoId },
    data: {
      title,
      completed,
      updatedAt: new Date(),
    },
  });

  revalidatePath('/todos');
  revalidatePath(`/todos/${todoId}`);
}

// Toggle completion status
export async function toggleTodo(todoId: string, completed: boolean) {
  await db.todo.update({
    where: { id: todoId },
    data: { completed, updatedAt: new Date() },
  });

  revalidatePath('/todos');
}
```

### Delete Operations

```tsx
// app/actions/todo.ts
'use server';

export async function deleteTodo(todoId: string) {
  await db.todo.delete({
    where: { id: todoId },
  });

  revalidatePath('/todos');
}
```

### Batch Operations

```tsx
// app/actions/todo.ts
'use server';

export async function bulkDeleteTodos(todoIds: string[]) {
  await db.todo.deleteMany({
    where: {
      id: { in: todoIds },
    },
  });

  revalidatePath('/todos');
}

export async function bulkUpdatePriority(todoIds: string[], priority: string) {
  await db.todo.updateMany({
    where: {
      id: { in: todoIds },
    },
    data: { priority },
  });

  revalidatePath('/todos');
}
```

### Complete Todo Application Example

```tsx
// app/todos/page.tsx
import { getTodos } from '@/lib/db';
import { TodoList } from '@/components/TodoList';
import { CreateTodoForm } from '@/components/CreateTodoForm';

export default async function TodosPage() {
  const todos = await getTodos();

  return (
    <div className="container mx-auto px-4 py-8 max-w-3xl">
      <h1 className="text-3xl font-bold mb-8">Todo List</h1>
      
      <div className="bg-white rounded-lg shadow-md p-6 mb-8">
        <h2 className="text-lg font-semibold mb-4">Add New Todo</h2>
        <CreateTodoForm />
      </div>

      <TodoList initialTodos={todos} />
    </div>
  );
}
```

```tsx
// components/TodoList.tsx
'use client';

import { useOptimistic } from 'react';
import { TodoItem } from './TodoItem';
import { createTodo, toggleTodo, deleteTodo } from '@/app/actions/todo';

interface Todo {
  id: string;
  title: string;
  completed: boolean;
  priority: string;
}

export function TodoList({ initialTodos }: { initialTodos: Todo[] }) {
  const [optimisticTodos, addOptimisticTodo] = useOptimistic(
    initialTodos,
    (state, newTodo: Todo) => [...state, newTodo]
  );

  async function handleCreate(formData: FormData) {
    const title = formData.get('title') as string;
    
    // Optimistic update
    addOptimisticTodo({
      id: `temp-${Date.now()}`,
      title,
      completed: false,
      priority: 'medium',
    });

    await createTodo(formData);
  }

  return (
    <div className="space-y-4">
      {optimisticTodos.map((todo) => (
        <TodoItem 
          key={todo.id} 
          todo={todo} 
          onToggle={toggleTodo}
          onDelete={deleteTodo}
        />
      ))}
    </div>
  );
}
```

```tsx
// components/TodoItem.tsx
'use client';

import { useTransition } from 'react';

interface TodoItemProps {
  todo: {
    id: string;
    title: string;
    completed: boolean;
    priority: string;
  };
  onToggle: (id: string, completed: boolean) => Promise<void>;
  onDelete: (id: string) => Promise<void>;
}

export function TodoItem({ todo, onToggle, onDelete }: TodoItemProps) {
  const [isPending, startTransition] = useTransition();

  return (
    <div className={`flex items-center justify-between p-4 bg-white rounded-lg shadow ${isPending ? 'opacity-50' : ''}`}>
      <div className="flex items-center space-x-4">
        <input
          type="checkbox"
          checked={todo.completed}
          onChange={(e) => {
            startTransition(() => {
              onToggle(todo.id, e.target.checked);
            });
          }}
          className="w-5 h-5 text-blue-600 rounded focus:ring-blue-500"
        />
        
        <div>
          <p className={`font-medium ${todo.completed ? 'line-through text-gray-500' : 'text-gray-900'}`}>
            {todo.title}
          </p>
          <span className={`text-xs px-2 py-1 rounded-full ${
            todo.priority === 'high' ? 'bg-red-100 text-red-800' :
            todo.priority === 'medium' ? 'bg-yellow-100 text-yellow-800' :
            'bg-green-100 text-green-800'
          }`}>
            {todo.priority}
          </span>
        </div>
      </div>

      <button
        onClick={() => startTransition(() => onDelete(todo.id))}
        className="text-red-600 hover:text-red-800"
      >
        Delete
      </button>
    </div>
  );
}
```

## 11.5 Revalidation After Mutations

One of the most powerful features of Server Actions is automatic cache revalidation after data changes.

### Basic Revalidation

```tsx
// app/actions/post.ts
'use server';

import { revalidatePath } from 'next/cache';

export async function createPost(formData: FormData) {
  await db.post.create({
    data: {
      title: formData.get('title'),
      content: formData.get('content'),
    },
  });

  // Revalidate the posts list page
  revalidatePath('/posts');
  
  // Also revalidate the home page if it shows recent posts
  revalidatePath('/');
}
```

### Tag-Based Revalidation

```tsx
// app/actions/post.ts
'use server';

import { revalidateTag } from 'next/cache';

export async function updatePost(postId: string, formData: FormData) {
  const post = await db.post.update({
    where: { id: postId },
    data: {
      title: formData.get('title'),
      content: formData.get('content'),
      categoryId: formData.get('categoryId'),
    },
  });

  // Revalidate specific post
  revalidateTag(`post-${postId}`);
  
  // Revalidate posts list
  revalidateTag('posts');
  
  // Revalidate category if changed
  if (post.categoryId) {
    revalidateTag(`category-${post.categoryId}`);
  }
}
```

### Conditional Revalidation

```tsx
// app/actions/settings.ts
'use server';

import { revalidatePath } from 'next/cache';

export async function updateUserSettings(userId: string, formData: FormData) {
  const oldSettings = await db.settings.findUnique({ where: { userId } });
  
  const newSettings = await db.settings.update({
    where: { userId },
    data: {
      theme: formData.get('theme'),
      language: formData.get('language'),
      notifications: formData.get('notifications') === 'true',
    },
  });

  // Only revalidate if theme changed (affects UI)
  if (oldSettings?.theme !== newSettings.theme) {
    revalidatePath('/dashboard');
    revalidatePath('/settings');
  }

  // Always revalidate settings page
  revalidatePath('/settings');
}
```

## 11.6 Error Handling in Server Actions

Robust error handling is crucial for Server Actions to provide good user feedback.

### Try-Catch Patterns

```tsx
// app/actions/user.ts
'use server';

export type ActionResult = {
  success: boolean;
  message: string;
  errors?: Record<string, string[]>;
};

export async function updateProfile(formData: FormData): Promise<ActionResult> {
  try {
    const userId = formData.get('userId') as string;
    const name = formData.get('name') as string;
    
    // Validation
    if (!name || name.length < 2) {
      return {
        success: false,
        message: 'Validation failed',
        errors: {
          name: ['Name must be at least 2 characters'],
        },
      };
    }

    // Database operation
    await db.user.update({
      where: { id: userId },
      data: { name },
    });

    return {
      success: true,
      message: 'Profile updated successfully',
    };
  } catch (error) {
    console.error('Update profile error:', error);
    
    // Handle specific error types
    if (error instanceof Error) {
      if (error.message.includes('Unique constraint')) {
        return {
          success: false,
          message: 'This name is already taken',
        };
      }
      
      if (error.message.includes('Record not found')) {
        return {
          success: false,
          message: 'User not found',
        };
      }
    }

    return {
      success: false,
      message: 'An unexpected error occurred. Please try again.',
    };
  }
}
```

### Global Error Handling

Create a wrapper for consistent error handling:

```tsx
// lib/server-actions.ts
'use server';

type ServerAction<T> = (formData: FormData) => Promise<T>;

export function withErrorHandling<T>(
  action: ServerAction<T>,
  errorMessage: string = 'Something went wrong'
): ServerAction<{ data?: T; error?: string }> {
  return async (formData: FormData) => {
    try {
      const result = await action(formData);
      return { data: result };
    } catch (error) {
      console.error('Server Action Error:', error);
      
      return {
        error: error instanceof Error ? error.message : errorMessage,
      };
    }
  };
}
```

### Client-Side Error Display

```tsx
// components/ErrorBoundary.tsx
'use client';

import { useState } from 'react';

export function ErrorDisplay({ error, reset }: { error: Error; reset: () => void }) {
  return (
    <div className="p-4 bg-red-50 border border-red-200 rounded-lg">
      <h3 className="text-red-800 font-semibold mb-2">Error</h3>
      <p className="text-red-600 mb-4">{error.message}</p>
      <button
        onClick={reset}
        className="px-4 py-2 bg-red-600 text-white rounded hover:bg-red-700"
      >
        Try Again
      </button>
    </div>
  );
}
```

## 11.7 Security Best Practices

Server Actions run on the server, but they still need security measures to prevent abuse and unauthorized access.

### Input Validation

Always validate and sanitize inputs:

```tsx
// app/actions/comment.ts
'use server';

import { z } from 'zod';
import DOMPurify from 'isomorphic-dompurify';

const commentSchema = z.object({
  postId: z.string().uuid(),
  content: z.string().min(1).max(1000),
  authorName: z.string().min(1).max(100),
});

export async function createComment(formData: FormData) {
  // Validate input
  const result = commentSchema.safeParse({
    postId: formData.get('postId'),
    content: formData.get('content'),
    authorName: formData.get('authorName'),
  });

  if (!result.success) {
    throw new Error('Invalid input');
  }

  // Sanitize HTML content
  const cleanContent = DOMPurify.sanitize(result.data.content);

  // Check rate limiting (example)
  const ip = headers().get('x-forwarded-for');
  if (await isRateLimited(ip)) {
    throw new Error('Too many requests. Please try again later.');
  }

  await db.comment.create({
    data: {
      ...result.data,
      content: cleanContent,
    },
  });
}
```

### Authentication Checks

Verify user permissions:

```tsx
// app/actions/admin.ts
'use server';

import { auth } from '@clerk/nextjs/server';

export async function deleteUser(userId: string) {
  // Check authentication
  const { userId: currentUserId, sessionClaims } = auth();
  
  if (!currentUserId) {
    throw new Error('Unauthorized');
  }

  // Check authorization (admin only)
  if (sessionClaims?.role !== 'admin') {
    throw new Error('Forbidden: Admin access required');
  }

  // Prevent self-deletion
  if (currentUserId === userId) {
    throw new Error('Cannot delete your own account');
  }

  await db.user.delete({ where: { id: userId } });
  revalidatePath('/admin/users');
}
```

### CSRF Protection

Next.js automatically handles CSRF protection for Server Actions, but be aware of cross-origin restrictions:

```tsx
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  // Check origin for sensitive actions
  const origin = request.headers.get('origin');
  const allowedOrigins = ['https://yoursite.com', 'https://app.yoursite.com'];
  
  if (origin && !allowedOrigins.includes(origin)) {
    return new NextResponse('Invalid origin', { status: 403 });
  }

  return NextResponse.next();
}

export const config = {
  matcher: ['/api/:path*', '/actions/:path*'],
};
```

### Rate Limiting

Implement rate limiting for Server Actions:

```tsx
// lib/rate-limit.ts
import { LRUCache } from 'lru-cache';

const rateLimitCache = new LRUCache({
  max: 500,
  ttl: 1000 * 60 * 15, // 15 minutes
});

export async function rateLimit(identifier: string, limit: number = 10) {
  const current = (rateLimitCache.get(identifier) as number) || 0;
  
  if (current >= limit) {
    throw new Error('Rate limit exceeded. Please try again later.');
  }
  
  rateLimitCache.set(identifier, current + 1);
}

// Usage in Server Action
export async function submitForm(formData: FormData) {
  const ip = headers().get('x-forwarded-for') || 'unknown';
  await rateLimit(ip, 5); // 5 requests per 15 minutes
  
  // Process form...
}
```

### Secure File Handling

When handling file uploads:

```tsx
// app/actions/upload.ts
'use server';

export async function uploadDocument(formData: FormData) {
  const file = formData.get('document') as File;
  
  // Validate file type strictly
  const allowedExtensions = ['.pdf', '.doc', '.docx'];
  const ext = '.' + file.name.split('.').pop()?.toLowerCase();
  
  if (!allowedExtensions.includes(ext)) {
    throw new Error('Invalid file type');
  }

  // Scan for malware (pseudo-code)
  const scanResult = await virusScan(file);
  if (!scanResult.clean) {
    throw new Error('File failed security scan');
  }

  // Store in secure location, not public
  const path = await saveToSecureStorage(file);
  
  return { success: true, path };
}
```

## Key Takeaways from Chapter 11

1. **Introduction to Server Actions**: Server Actions allow you to run server-side code directly from components using the `'use server'` directive, eliminating the need for separate API routes and reducing boilerplate while maintaining type safety.

2. **Creating Server Actions**: Server Actions can be defined in separate files with `'use server'` at the top, or inline within Server Components. They can accept FormData, regular parameters, and use closures to bind additional data.

3. **Form Handling**: Server Actions provide progressive enhancement for forms, working without JavaScript but enhanced with it. Use `useFormState` and `useFormStatus` hooks for managing form submission states and validation.

4. **Mutation Patterns**: Implement Create, Read, Update, Delete (CRUD) operations using Server Actions with proper validation using Zod or similar libraries. Support batch operations and optimistic updates for better UX.

5. **Revalidation After Mutations**: Use `revalidatePath()` to invalidate specific routes or `revalidateTag()` for tag-based invalidation after mutations, ensuring the UI stays in sync with data changes.

6. **Error Handling**: Implement comprehensive try-catch blocks, return structured error responses, use global error handling wrappers, and display user-friendly error messages on the client.

7. **Security Best Practices**: Always validate and sanitize inputs using Zod, check authentication and authorization, implement rate limiting, handle file uploads securely, and leverage Next.js's built-in CSRF protection.

## Coming Up Next

**Chapter 12: Route Handlers & APIs**

Now that you understand Server Actions for mutations, it's time to learn about Route Handlers for creating RESTful APIs and handling more complex server-side logic. In Chapter 12, we'll explore creating API routes, handling different HTTP methods, request/response objects, CORS configuration, and building robust backend endpoints for your Next.js applications.